In [1]:
# importing libraries for data preprocessing
import pandas as pd
import numpy as np

# importing libraries for text processing
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# importing libraries for building classifier model
from nltk.sentiment import SentimentIntensityAnalyzer
from sklearn.naive_bayes import (
    BernoulliNB,
    ComplementNB,
    MultinomialNB,
)
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

In [2]:
# importing the data
df = pd.read_csv('Information.csv')

In [3]:
df.head()

,_unit_id,_golden,_unit_state,_trusted_judgments,_last_judgment_at,gender,gender:confidence,profile_yn,profile_yn:confidence,created,...,profileimage,retweet_count,sidebar_color,text,tweet_coord,tweet_count,tweet_created,tweet_id,tweet_location,user_timezone
0,815719226,False,finalized,3,10/26/15 23:24,male,1.0000,yes,1.0,12/5/13 1:48,...,https://pbs.twimg.com/profile_images/414342229...,0,FFFFFF,Robbie E Responds To Critics After Win Against...,NaN,110964,10/26/15 12:40,6.587300e+17,main; @Kan1shk3,Chennai
1,815719227,False,finalized,3,10/26/15 23:30,male,1.0000,yes,1.0,10/1/12 13:51,...,https://pbs.twimg.com/profile_images/539604221...,0,C0DEED,‰ÛÏIt felt like they were my friends and I was...,NaN,7471,10/26/15 12:40,6.587300e+17,NaN,Eastern Time (US & Canada)
2,815719228,False,finalized,3,10/26/15 23:33,male,0.6625,yes,1.0,11/28/14 11:30,...,https://pbs.twimg.com/profile_images/657330418...,1,C0DEED,i absolutely adore when louis starts the songs...,NaN,5617,10/26/15 12:40,6.587300e+17,clcncl,Belgrade
3,815719229,False,finalized,3,10/26/15 23:10,male,1.0000,yes,1.0,6/11/09 22:39,...,https://pbs.twimg.com/profile_images/259703936...,0,C0DEED,Hi @JordanSpieth - Looking at the url - do you...,NaN,1693,10/26/15 12:40,6.587300e+17,"Palo Alto, CA",Pacific Time (US & Canada)
4,815719230,False,finalized,3,10/27/15 1:15,female,1.0000,yes,1.0,4/16/14 13:23,...,https://pbs.twimg.com/profile_images/564094871...,0,0,Watching Neighbours on Sky+ catching up with t...,NaN,31462,10/26/15 12:40,6.587300e+17,NaN,NaN


In [4]:
df.shape

(20050, 26)

In [5]:
# dropping the rows that contain null values in the 'text' column.
df.dropna(subset=['text'])

,_unit_id,_golden,_unit_state,_trusted_judgments,_last_judgment_at,gender,gender:confidence,profile_yn,profile_yn:confidence,created,...,profileimage,retweet_count,sidebar_color,text,tweet_coord,tweet_count,tweet_created,tweet_id,tweet_location,user_timezone
0,815719226,False,finalized,3,10/26/15 23:24,male,1.0000,yes,1.0,12/5/13 1:48,...,https://pbs.twimg.com/profile_images/414342229...,0,FFFFFF,Robbie E Responds To Critics After Win Against...,NaN,110964,10/26/15 12:40,6.587300e+17,main; @Kan1shk3,Chennai
1,815719227,False,finalized,3,10/26/15 23:30,male,1.0000,yes,1.0,10/1/12 13:51,...,https://pbs.twimg.com/profile_images/539604221...,0,C0DEED,‰ÛÏIt felt like they were my friends and I was...,NaN,7471,10/26/15 12:40,6.587300e+17,NaN,Eastern Time (US & Canada)
2,815719228,False,finalized,3,10/26/15 23:33,male,0.6625,yes,1.0,11/28/14 11:30,...,https://pbs.twimg.com/profile_images/657330418...,1,C0DEED,i absolutely adore when louis starts the songs...,NaN,5617,10/26/15 12:40,6.587300e+17,clcncl,Belgrade
3,815719229,False,finalized,3,10/26/15 23:10,male,1.0000,yes,1.0,6/11/09 22:39,...,https://pbs.twimg.com/profile_images/259703936...,0,C0DEED,Hi @JordanSpieth - Looking at the url - do you...,NaN,1693,10/26/15 12:40,6.587300e+17,"Palo Alto, CA",Pacific Time (US & Canada)
4,815719230,False,finalized,3,10/27/15 1:15,female,1.0000,yes,1.0,4/16/14 13:23,...,https://pbs.twimg.com/profile_images/564094871...,0,0,Watching Neighbours on Sky+ catching up with t...,NaN,31462,10/26/15 12:40,6.587300e+17,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20045,815757572,True,golden,259,NaN,female,1.0000,yes,1.0,8/5/15 21:16,...,https://pbs.twimg.com/profile_images/656793310...,0,C0DEED,"@lookupondeath ...Fine, and I'll drink tea too...",NaN,783,10/26/15 13:20,6.587400e+17,Verona ‰ªÁ,NaN
20046,815757681,True,golden,248,NaN,male,1.0000,yes,1.0,8/15/12 21:17,...,https://pbs.twimg.com/profile_images/639815429...,0,0,Greg Hardy you a good player and all but don't...,NaN,13523,10/26/15 12:40,6.587300e+17,"Kansas City, MO",NaN
20047,815757830,True,golden,264,NaN,male,1.0000,yes,1.0,9/3/12 1:17,...,https://pbs.twimg.com/profile_images/655473271...,0,C0DEED,You can miss people and still never want to se...,NaN,26419,10/26/15 13:20,6.587400e+17,Lagos Nigeria,NaN
20048,815757921,True,golden,250,NaN,female,0.8489,yes,1.0,11/6/12 23:46,...,https://pbs.twimg.com/profile_images/657716093...,0,0,@bitemyapp i had noticed your tendency to pee ...,NaN,56073,10/26/15 12:40,6.587300e+17,Texas Hill Country,NaN


In [6]:
# assigning stopwords 
stop = stopwords.words('english')

# breaking down the sentences in the text column into a list of words
df['text'] = df['text'].apply(lambda x:word_tokenize(x))
# excluding the stopwords
df['text'] = df['text'].apply(lambda y:[x for x in y if x.lower() not in stop])

In [7]:
# creating a consolidated list of words from each row.
words = []
for x in df['text']:
    for y in x:
        if y.isalpha():
            words.append(y)    

In [8]:
words = [x.lower() for x in words if x.lower() not in stop]
words

['robbie',
 'e',
 'responds',
 'critics',
 'win',
 'eddie',
 'edwards',
 'worldtitleseries',
 'https',
 'felt',
 'like',
 'friends',
 'living',
 'story',
 'https',
 'retired',
 'https',
 'absolutely',
 'adore',
 'louis',
 'starts',
 'songs',
 'hits',
 'hard',
 'feels',
 'good',
 'hi',
 'jordanspieth',
 'looking',
 'url',
 'use',
 'ifttt',
 'typically',
 'see',
 'advanced',
 'user',
 'pgatour',
 'https',
 'watching',
 'neighbours',
 'catching',
 'neighbs',
 'xxx',
 'xxx',
 'ive',
 'seen',
 'people',
 'train',
 'lamps',
 'chairs',
 'tvs',
 'etc',
 'https',
 'bpackengineer',
 'thank',
 'patience',
 'take',
 'care',
 'issue',
 'gala',
 'bingo',
 'clubs',
 'bought',
 'uk',
 'largest',
 'high',
 'street',
 'bingo',
 'operator',
 'gala',
 'taken',
 'https',
 'pic',
 'defines',
 'mcd',
 'mcd',
 'shippers',
 'xd',
 'evielady',
 'lovely',
 'tree',
 'year',
 'never',
 'seen',
 'gorgeous',
 'autumn',
 'colour',
 'taking',
 'care',
 'https',
 'fitfluential',
 'mtg',
 'deals',
 'seventh',
 'one',
 '

In [9]:
# instance of Frequency Distribution class
fd = nltk.FreqDist(words)

In [10]:
# the most common words in the given data
fd.most_common(5)

[('https', 8477),
 ('weather', 2708),
 ('get', 2264),
 ('channel', 1385),
 ('updates', 1357)]

In [11]:
# joining the words for sentiment analysis
df['text'] = df['text'].apply(lambda x:' '.join(x))

In [12]:
# creating an instance of SentimentIntensityAnalyzer for finding the polarityscores
sia = SentimentIntensityAnalyzer()

# list for storing the scores
scores = []
for x in df['text']:
    score = sia.polarity_scores(x)
    scores.append(score)

In [13]:
# comverting the scores list to a dataframe and appending it to the df
scores = pd.DataFrame(scores)

df1 = pd.concat([df,scores],axis=1)

In [14]:
# dataframe after appending scores
df1.head()

,_unit_id,_golden,_unit_state,_trusted_judgments,_last_judgment_at,gender,gender:confidence,profile_yn,profile_yn:confidence,created,...,tweet_coord,tweet_count,tweet_created,tweet_id,tweet_location,user_timezone,neg,neu,pos,compound
0,815719226,False,finalized,3,10/26/15 23:24,male,1.0000,yes,1.0,12/5/13 1:48,...,NaN,110964,10/26/15 12:40,6.587300e+17,main; @Kan1shk3,Chennai,0.169,0.538,0.292,0.3818
1,815719227,False,finalized,3,10/26/15 23:30,male,1.0000,yes,1.0,10/1/12 13:51,...,NaN,7471,10/26/15 12:40,6.587300e+17,NaN,Eastern Time (US & Canada),0.000,0.663,0.337,0.6808
2,815719228,False,finalized,3,10/26/15 23:33,male,0.6625,yes,1.0,11/28/14 11:30,...,NaN,5617,10/26/15 12:40,6.587300e+17,clcncl,Belgrade,0.099,0.423,0.479,0.7501
3,815719229,False,finalized,3,10/26/15 23:10,male,1.0000,yes,1.0,6/11/09 22:39,...,NaN,1693,10/26/15 12:40,6.587300e+17,"Palo Alto, CA",Pacific Time (US & Canada),0.152,0.848,0.000,-0.3235
4,815719230,False,finalized,3,10/27/15 1:15,female,1.0000,yes,1.0,4/16/14 13:23,...,NaN,31462,10/26/15 12:40,6.587300e+17,NaN,NaN,0.000,1.000,0.000,0.0000


In [15]:
# creating a list and assiging 'positive,'negative' and 'neutral' values based on compound
sentiment = []
for x in df1.compound:
    if x >0:
        res = 'positive'
    elif x == 0:
        res = 'neutral'
    else:
        res = 'negative'
    sentiment.append(res)

In [16]:
df1['sentiment'] = sentiment

In [17]:
df1['sentiment']

0        positive
1        positive
2        positive
3        negative
4         neutral
           ...   
20045    positive
20046    positive
20047    negative
20048    negative
20049    positive
Name: sentiment, Length: 20050, dtype: object

In [18]:
def text(x):
    return {'text':x}

In [19]:
# creating featureset for input in classifier
featureset = [(text(i),j) for i,j in zip(df1['text'].values,df1['sentiment'].values)]

In [20]:
# splitting into training and testing set
train, test = featureset[:18000],featureset[18000:]

In [21]:
classifier = nltk.NaiveBayesClassifier.train(train)

In [22]:
nltk.classify.accuracy(classifier,test)

0.4473170731707317

In [23]:
# sklearn classifiers to incorporate in nltk 
classifiers = {
    "BernoulliNB": BernoulliNB(),
    "ComplementNB": ComplementNB(),
    "MultinomialNB": MultinomialNB(),
    "KNeighborsClassifier": KNeighborsClassifier(),
    "DecisionTreeClassifier": DecisionTreeClassifier(),
    "RandomForestClassifier": RandomForestClassifier(),
    "LogisticRegression": LogisticRegression(),
    "MLPClassifier": MLPClassifier(max_iter=1000),
    "AdaBoostClassifier": AdaBoostClassifier(),
}

In [24]:
features=[]
for x in range(0,len(df1['text'])):
    if df1['sentiment'][x]  == 'positive':
        features.append((text(df1['text'][x]),'positive'))
    elif df1['sentiment'][x] == 'neutral':
        features.append((text(df1['text'][x]),'neutral'))
    else:
        features.append((text(df1['text'][x]),'negative'))

In [25]:
# training the model and calculating accuracy for each classifier used
train_count = len(features) // 4
for name, sklearn_classifier in classifiers.items():
    classifier = nltk.classify.SklearnClassifier(sklearn_classifier)
    classifier.train(features[:train_count*3])
    accuracy = nltk.classify.accuracy(classifier, features[train_count:])
    print(F"{accuracy:.2%} - {name}")

68.34% - BernoulliNB
75.02% - ComplementNB
82.09% - MultinomialNB
60.08% - KNeighborsClassifier
82.09% - DecisionTreeClassifier
82.03% - RandomForestClassifier
82.09% - LogisticRegression
82.09% - MLPClassifier
51.60% - AdaBoostClassifier


### It is obvious that the classifiers MultinomialNB, DecisionTreeClassifier, LogisticRegression, MLPClassifier performs really well with an accuracy of 82.09%.